# FOM Optimization

We use a figure of merit (FOM) value as an optimization parameter for training our classification. The FOM is defined as:

$$FOM = \frac{N_{true}}{N_{tot}} * \frac{N_{true}}{N_{true} + N_{false}}$$

where $N_{true}$ is the number of objects correctly classified as a given type (e.g. 91bg-like objects), $N_{tot}$ is the total number of that type, and $N_{false}$ is the number of incorrectly classified objects. 


In [ ]:
def subplot_fom_boundary(fom_type, axis, *args, **kwargs):
    """Plot the boundaries from an FOM calculation
    
    Args:
        fom_type (str): The type of FOM calculation that was used 
        axis    (Axis): The matplotlib axis to plot on 
        *args  (float): The boundaries of the FOM calculation  
        **kwargs: Plotting options 
    """
    
    xlim = axis.get_xlim()
    ylim = axis.get_ylim()
    if fom_type == 'rectangular':
        axis.axhline(args[0], **kwargs)
        if 'label' in kwargs:
            kwargs.pop('label')

        axis.axvline(args[1], **kwargs)
        
    elif fom_type == 'horizontal':
        axis.axvline(args[0], **kwargs)

    elif fom_type == 'vertical':
        axis.axhline(args[0], **kwargs)

    elif fom_type == 'linear':
        x = np.array([-1e4, 1e4])
        axis.plot(x, args[0] * x + args[1], **kwargs)

    elif fom_type == 'diagonal':
        x = np.array([-1e4, 1e4])
        axis.plot(x, - x + args[0], **kwargs)

    else:
        raise ValueError(f'Unknown FOM type {fom_type}')

    axis.set_xlim(xlim)
    axis.set_ylim(ylim)
    axis.legend()


In [ ]:
static_fig, static_axis = create_static_figure(classifications)

rectangular_lam = lambda args: 1 - fom.rectangular(truth=classifications['spec_class'],
    x=classifications['x'], 
    y=classifications['y'], 
    x_cutoff=args[0], 
    y_cutoff=args[1], 
    check_type='91bg')

rectangular_min = optimize.minimize(rectangular_lam, np.array([0, 0]), method='Powell')
x_cutoff, y_cutoff = rectangular_min['x']
rectangular_fom = 1 - rectangular_min['fun']

subplot_fom_boundary(
    'rectangular', 
    static_axis, 
    x_cutoff, 
    y_cutoff,
    linestyle='--', 
    color='black', 
    alpha=.6, 
    label=f'FOM = {rectangular_fom:.3}')

plt.show()

In [ ]:
static_fig, static_axis = create_static_figure(classifications)

vertical_lam = lambda args: 1 - fom.vertical(truth=classifications['spec_class'],
    y=classifications['y'], 
    y_cutoff=args[0], 
    check_type='91bg')

vertical_min = optimize.minimize(vertical_lam, np.array([0]), method='Powell')
vertical_cutoff = vertical_min['x']
vertical_fom = 1 - vertical_min['fun']

subplot_fom_boundary(
    'vertical', 
    static_axis, 
    vertical_cutoff, 
    linestyle='--',
    color='black',
    alpha=.6, 
    label=f'FOM = {vertical_fom:.3}')

horizontal_lam = lambda args: 1 - fom.horizontal(truth=classifications['spec_class'],
    x=classifications['x'], 
    x_cutoff=args[0], 
    check_type='91bg')

horizontal_min = optimize.minimize(horizontal_lam, np.array([0]), method='Powell')
horizontal_cutoff = horizontal_min['x']
horizontal_fom = 1 - horizontal_min['fun']

subplot_fom_boundary(
    'horizontal', 
    static_axis, 
    horizontal_cutoff, 
    linestyle=':',
    color='black',
    alpha=.6, 
    label=f'FOM = {horizontal_fom:.3}')


In [ ]:
static_fig, static_axis = create_static_figure(classifications)

diagonal_lam = lambda args: 1 - fom.diagonal(truth=classifications['spec_class'],
    x=classifications['x'], 
    y=classifications['y'],
    b=args[0],
    check_type='91bg')

diagonal_min = optimize.minimize(diagonal_lam, np.array([0]), method='Powell')
diagonal_b = diagonal_min['x']
diagonal_fom = 1 - diagonal_min['fun']

subplot_fom_boundary(
    'diagonal', 
    static_axis, 
    diagonal_b, 
    linestyle='--', 
    color='black', 
    alpha=.6,
    label=f'Diagonal FOM ({diagonal_b:.2}) = {diagonal_fom:.3}')

linear_lam = lambda args: 1 - fom.linear(truth=classifications['spec_class'],
    x=classifications['x'], 
    y=classifications['y'],
    m=args[0],
    b=args[1],
    check_type='91bg')

linear_min = optimize.minimize(linear_lam, np.array([-5, 0]), method='Powell')
linear_m, linear_b = linear_min['x']
linear_fom = 1 - linear_min['fun']

subplot_fom_boundary(
    'diagonal', 
    static_axis, 
    linear_m, 
    linear_b, 
    linestyle=':', 
    color='black', 
    alpha=.6,
    label=f'Linear FOM ({linear_m:.2}, {linear_b:.2})= {linear_fom:.4}')


## Bootstrap

Now that we have a figure of merit optimization, we can bootstrap our data to determine our final classification parameters.

In [ ]:
# configure bootstrap
n_iterations = 100
n_size = int(len(classifications) * 0.50)

# run bootstrap
fom_values = []
classification_params = []
for i in range(n_iterations):
    # prepare train and test sets
    sample_data = resample(classifications, n_samples=n_size)
    result = optimize.minimize(rectangular_lam, [0, 0], method='Powell')

    fom_values.append(1 - result.fun)
    classification_params.append(result.x)

classification_params = np.array(classification_params).T


In [ ]:
def calc_confidence_intervals(alpha, stats):
    p = ((1 - alpha) / 2) * 100
    lower = max(0, np.percentile(stats, p))

    p = (alpha + ((1 - alpha) / 2)) * 100
    upper = min(1.0, np.percentile(stats, p))

    return lower, upper


alpha=0.95
confidence = 0.95
average_fom = np.average(fom_values)
fom_interval = calc_confidence_intervals(confidence, fom_values)

print(f'Average FOM: {average_fom}')
print(f'{alpha * 100:.1f} confidence interval: [{fom_interval[0] * 100:.1f} '
      f', {fom_interval[1] * 100:.1f}]')

average_params = np.average(classification_params, axis=1)
blue_param_interval = calc_confidence_intervals(confidence, classification_params[0])
red_param_interval = calc_confidence_intervals(confidence, classification_params[1])

print(f'Average classification params: {average_params}')
print(f'{alpha * 100} confidence interval for blue '
      f'boundary: [{blue_param_interval[0]:.2f} '
      f', {blue_param_interval[1]:.2f}]')

print(f'{alpha * 100} confidence interval for red '
      f'boundary: [{red_param_interval[0]:.2f} '
      f', {red_param_interval[1]:.2f}]')
